## This notebook imports slk and saves to .csv for easier access. 
note. SLK is read using sylk_parser and saved as text file. Text files are then read back in, coordinates are converted to numeric, and H:M:S timestamps are converted into seconds. The new dataframe with numeric coords and time in seconds is saved back to orginial data folder. 

Also plots the paths and saves to folder. 
Berkowitz 06/2021

In [1]:
# file structures and io
import glob
import os
from io import StringIO
import csv
from sylk_parser import SylkParser

# data wrangling 
import pandas as pd

# plotting 
import matplotlib.pyplot as plt

In [2]:
def slk_to_text(file,path):
    """ convert anymaze slk to txt, and saves back to file location"""
    parser = SylkParser(path+file+'.slk')
    fbuf = StringIO()
    parser.to_csv(fbuf)
    test_results = fbuf.getvalue()
    # write text document
    f = open(path+file+'.txt', 'w')
    f.write(test_results)
    f.close()
    
def get_sec(times):
    """Get Seconds from time."""
    seconds = []
    for time_str in times:
        h, m, s = time_str.split(':')
        seconds.append(pd.to_numeric(h) * 3600 + pd.to_numeric(m) * 60 + pd.to_numeric(s))
    return seconds

def read_anymaze_txt(file_path,dest_path):
    """
    reads text file of anymaze data and saves xy coordinates as numpy arrays
    """
    df = pd.read_csv(file_path)
    # convert coordinates to numeric
    x = pd.to_numeric(df['Centre position X'][1:]) # start at index 1 given anymaze doesn't provide coordinate for time at 0 sec
    y = pd.to_numeric(df['Centre position Y'][1:]) # start at index 1 given anymaze doesn't provide coordinate for time at 0 sec
    # convert H:M:S timestamps to seconds
    seconds = get_sec(df['Time'])
    # Create new df 
    new_df = pd.DataFrame()
    new_df['ts'] = seconds[1:]
    new_df['x'] = x.values
    new_df['y'] = y.values
    new_df.to_csv(dest_path+os.path.split(file_path)[-1][:-4]+'.csv',index=False)

## Save .slk files as text files

In [179]:
files = glob.glob("H:/OF/**/*.slk")

for file in files: 
    path = os.path.split(file)[0]
    name = os.path.split(file)[-1]
    slk_to_text(name[:-4],path+"/")

## Read in .txt file and save coordinates as csv

In [180]:
files = glob.glob("H:/OF/**/*.txt")

for file in files: 
    # this function converts strings to 
    read_anymaze_txt(file,'H:/OF/coords_csv/')


## Plot paths and save

In [21]:
files = glob.glob("H:/OF/coords_csv/*.csv")

for file in files: 
    df = pd.read_csv(file)
    # this function converts strings to 
    fig = plt.figure(figsize = (7,7))
    plt.plot(df.x,df.y)
    plt.savefig('H:/OF/figures/paths/' + file.split('\\')[1] +'.tiff',dpi=300,bbox_inches='tight')
    plt.close()
